<a href="https://colab.research.google.com/github/aazat3/MindBox3/blob/main/MindBox_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=0744811c0d19ae5457ca1dbd3e43da3f942505be0ff506b1033a1c1208621bd1
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat_ws


def get_product_category_pairs_with_no_category(products_df, categories_df, products_categories_data):
    # Выполняем объединение продуктов и категорий по соответствующему столбцу
    products_with_categories_df = products_df\
        .join(products_categories_data, products_df["product_id"] == products_categories_data["product_id"], "inner")\
        .join(categories_df, products_categories_data["category_id"] == categories_df["category_id"], "inner")\
        .select(products_df["product_name"], categories_df["category_name"])

    # Выполняем объединение имени продукта и имени категории
    products_categories_df = products_with_categories_df.withColumn("product_category", concat_ws("-" ,col("product_name"), col("category_name")))
    products_categories_df = products_categories_df.select(products_categories_df["product_category"])


    # Ищем продукты без категории
    products_with_no_category_df = products_df\
        .join(products_categories_data, products_df["product_id"] == products_categories_data["product_id"], "leftanti")\
        .select(products_df["product_name"])

    # Объединяем датафреймы
    result_df = products_categories_df.union(products_with_no_category_df)

    return result_df


# Пример использования:
if __name__ == "__main__":
    spark = SparkSession.builder \
        .appName("ProductCategoryPairs") \
        .getOrCreate()

    # Пример данных для продуктов
    products_data = [(1, "product1"),
                     (2, "product2"),
                     (3, "product3")]

    # Пример данных для категорий
    categories_data = [(1, "category1"),
                       (2, "category2"),
                       (3, "category3")]

    # Пример данных для связи продукта и категории (из-за связи многие ко многим)
    products_categories_data = [(1, 1),
                                (1, 2),
                                (2, 2)]

    products_df = spark.createDataFrame(products_data, ["product_id", "product_name"])
    categories_df = spark.createDataFrame(categories_data, ["category_id", "category_name"])
    products_categories_data = spark.createDataFrame(products_categories_data, ["product_id", "category_id"])

    result_df = get_product_category_pairs_with_no_category(products_df, categories_df, products_categories_data)
    result_df.show()

    spark.stop()


+------------------+
|  product_category|
+------------------+
|product1-category1|
|product2-category2|
|product1-category2|
|          product3|
+------------------+

